### Ingest data on Google Earth Engine (WRI bucket)

* Purpose of script: This notebook will upload the geotiff files from the Google Cloud Storage to the WRI/aqueduct earthengine bucket. 
* Author: Rutger Hofste
* Kernel used: python36
* Date created: 20170802

## Preparation

* Authorize earthengine by running in your terminal: `earthengine authenticate`  
* you need to have access to the WRI-Aquaduct (yep a Google employee made a typo) bucket to ingest the data. Rutger can grant access to write to this folder. 
* Have access to the Google Cloud Storage Bucker

Run in your terminal `gcloud config set project aqueduct30`

In [85]:
import subprocess
import datetime
import os
import time
from datetime import timedelta
import re
import pandas as pd

## Settings

In [2]:
GCS_BASE = "gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/"

In [3]:
EE_BASE = "projects/WRI-Aquaduct/PCRGlobWB20V05"

## Functions

In [4]:
def splitKey(key):
    # will yield the root file code and extension of a set of keys
    prefix, extension = key.split(".")
    fileName = prefix.split("/")[-1]
    parameter = fileName[:-12]
    month = fileName[-2:] #can also do this with regular expressions if you like
    year = fileName[-7:-3]
    identifier = fileName[-11:-8]
    outDict = {"fileName":fileName,"extension":extension,"parameter":parameter,"month":month,"year":year,"identifier":identifier}
    return outDict

def splitParameter(parameter):
    #values = parameter.split("_")
    values = re.split("_|-", parameter) #soilmoisture uses a hyphen instead of underscore between the years
    keys = ["geographic_range","temporal_range","indicator","temporal_resolution","units","spatial_resolution","temporal_range_min","temporal_range_max"]
    # ['global', 'historical', 'PDomWN', 'month', 'millionm3', '5min', '1960', '2014']
    outDict = dict(zip(keys, values))
    outDict["parameter"] = parameter
    return outDict


## Script

In [5]:
command = ("earthengine create folder %s") %EE_BASE

In [6]:
print(command)

earthengine create folder projects/WRI-Aquaduct/PCRGlobWB20V05


In [7]:
subprocess.check_output(command,shell=True)

'Asset projects/WRI-Aquaduct/PCRGlobWB20V05 already exists\n'

In [8]:
command = ("/opt/google-cloud-sdk/bin/gsutil ls %s") %(GCS_BASE)

In [9]:
keys = subprocess.check_output(command,shell=True)

In [10]:
keys = keys.decode('UTF-8').splitlines()

Removing first item from the list. The first item contains a folder without file name

In [11]:
keys2 = keys[1:]

In [12]:
df = pd.DataFrame()
i = 0
for key in keys2:
    i = i+1
    outDict = splitKey(key)
    df2 = pd.DataFrame(outDict,index=[i])
    df = df.append(df2)    

In [13]:
df.head()

,extension,fileName,identifier,month,parameter,year
1,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960
2,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960
3,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960
4,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960
5,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960


In [14]:
df.tail()

,extension,fileName,identifier,month,parameter,year
8545,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014
8546,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014
8547,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014
8548,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014
8549,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014


In [15]:
df.shape

(8549, 6)

In [16]:
parameters = df.parameter.unique()

In [17]:
print(parameters)

[u'global_historical_PDomWN_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWN_year_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_month_millionm3_5min_1960_2014'
 u'global_historical_PDomWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIndWW_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWN_year_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_month_millionm3_5min_1960_2014'
 u'global_historical_PIrrWW_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWN_year_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_month_millionm3_5min_1960_2014'
 u'global_historical_PLivWW_year_millionm3_5min_1960_2014'
 u'global_historical_aqbasinwaterstress_month_di

We will store the geotiff images of each NetCDF4 file in imageCollections. The imageCollections will have the same name and content as the original NetCDF4files. 


In [19]:
for parameter in parameters:
    eeLocation = EE_BASE + "/" + parameter
    command = ("earthengine create collection %s") %eeLocation
    # Uncomment the following command if you run this script for the first time
    #subprocess.check_output(command,shell=True)
    #print(command)
    

Now that the folder and collections have been created we can start ingesting the data. It is crucial to store the relevant metadata with the images. 

In [20]:
df_parameter = pd.DataFrame()
i = 0
for parameter in parameters:
    i = i+1
    outDict_parameter = splitParameter(parameter)
    df_parameter2 = pd.DataFrame(outDict_parameter,index=[i])
    df_parameter = df_parameter.append(df_parameter2)   
    

In [21]:
df_parameter.head()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
1,global,PDomWN,global_historical_PDomWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
2,global,PDomWN,global_historical_PDomWN_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
3,global,PDomWW,global_historical_PDomWW_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3
4,global,PDomWW,global_historical_PDomWW_year_millionm3_5min_1...,5min,historical,2014,1960,year,millionm3
5,global,PIndWN,global_historical_PIndWN_month_millionm3_5min_...,5min,historical,2014,1960,month,millionm3


In [22]:
df_parameter.tail()

,geographic_range,indicator,parameter,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units
19,global,griddedwaterstress,global_historical_griddedwaterstress_month_dim...,5min,historical,2014,1960,month,dimensionless
20,global,griddedwaterstress,global_historical_griddedwaterstress_year_dime...,5min,historical,2014,1960,year,dimensionless
21,global,riverdischarge,global_historical_riverdischarge_month_m3secon...,5min,historical,2014,1960,month,m3second
22,global,riverdischarge,global_historical_riverdischarge_year_m3second...,5min,historical,2014,1960,year,m3second
23,global,soilmoisture,global_historical_soilmoisture_month_meter_5mi...,5min,historical,2014,1958,month,meter


In [23]:
df_parameter.shape

(23, 9)

In [24]:
df_complete = df.merge(df_parameter,how='left',left_on='parameter',right_on='parameter')

Adding NoData value, ingested_by and exportdescription

In [72]:
df_complete["nodata"] = -9999
df_complete["ingested_by"] ="RutgerHofste"
df_complete["exportdescription"] = df_complete["indicator"] + "_" + df_complete["temporal_resolution"]+"Y"+df_complete["year"]+"M"+df_complete["month"]

In [73]:
df_complete.head()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
0,tif,global_historical_PDomWN_month_millionm3_5min_...,000,01,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M01
1,tif,global_historical_PDomWN_month_millionm3_5min_...,001,02,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M02
2,tif,global_historical_PDomWN_month_millionm3_5min_...,002,03,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M03
3,tif,global_historical_PDomWN_month_millionm3_5min_...,003,04,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M04
4,tif,global_historical_PDomWN_month_millionm3_5min_...,004,05,global_historical_PDomWN_month_millionm3_5min_...,1960,global,PDomWN,5min,historical,2014,1960,month,millionm3,-9999,RutgerHofste,PDomWN_monthY1960M05


In [74]:
df_complete.tail()

,extension,fileName,identifier,month,parameter,year,geographic_range,indicator,spatial_resolution,temporal_range,temporal_range_max,temporal_range_min,temporal_resolution,units,nodata,ingested_by,exportdescription
8544,tif,global_historical_soilmoisture_month_meter_5mi...,679,08,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M08
8545,tif,global_historical_soilmoisture_month_meter_5mi...,680,09,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M09
8546,tif,global_historical_soilmoisture_month_meter_5mi...,681,10,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M10
8547,tif,global_historical_soilmoisture_month_meter_5mi...,682,11,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M11
8548,tif,global_historical_soilmoisture_month_meter_5mi...,683,12,global_historical_soilmoisture_month_meter_5mi...,2014,global,soilmoisture,5min,historical,2014,1958,month,meter,-9999,RutgerHofste,soilmoisture_monthY2014M12


In [75]:
list(df_complete.columns.values)

['extension',
 'fileName',
 'identifier',
 'month',
 'parameter',
 'year',
 'geographic_range',
 'indicator',
 'spatial_resolution',
 'temporal_range',
 'temporal_range_max',
 'temporal_range_min',
 'temporal_resolution',
 'units',
 'nodata',
 'ingested_by',
 'exportdescription']

In [76]:
row = df_complete.loc[100]

In [95]:
def uploadEE(row):
    target = EE_BASE +"/"+ row.parameter + "/" + row.fileName
    source = GCS_BASE + row.fileName + "." + row.extension
    metadata = "--nodata_value=%s --time_start %s-%s-01 -p extension=%s -p filename=%s -p identifier=%s -p year=%s -p geographic_range=%s -p indicator=%s -p spatial_resolution=%s -p temporal_range=%s -p temporal_range_max=%s -p temporal_range_min=%s -p temporal_resolution=%s -p units=%s -p ingested_by=%s -p exportdescription=%s" %(row.nodata,row.year,row.month,row.extension,row.fileName,row.identifier,row.year,row.geographic_range,row.indicator,row.spatial_resolution,row.temporal_range,row.temporal_range_max,row.temporal_range_min, row.temporal_resolution, row.units, row.ingested_by, row.exportdescription)
    command = "/opt/anaconda3/bin/earthengine upload image --asset_id %s %s %s" % (target, source,metadata)
    try:
        subprocess.check_output(command, shell=True)
    except:
        print("ERROR!!")
        print(command)
    return 1



In [ ]:
start_time = time.time()
for index, row in df_complete.iterrows():
    elapsed_time = time.time() - start_time 
    print(index,"%.2f" %((index/8548.0)*100), "elapsed: ", str(timedelta(seconds=elapsed_time)))
    uploadEE(row)
    
    
    

(0, '0.00', 'elapsed: ', '0:00:00.003431')
(1, '0.01', 'elapsed: ', '0:00:01.290459')
(2, '0.02', 'elapsed: ', '0:00:02.450219')
(3, '0.04', 'elapsed: ', '0:00:03.624940')
(4, '0.05', 'elapsed: ', '0:00:05.089276')
(5, '0.06', 'elapsed: ', '0:00:06.275719')
(6, '0.07', 'elapsed: ', '0:00:07.455653')
(7, '0.08', 'elapsed: ', '0:00:08.643919')
(8, '0.09', 'elapsed: ', '0:00:09.935281')
(9, '0.11', 'elapsed: ', '0:00:11.083145')
(10, '0.12', 'elapsed: ', '0:00:12.310036')
(11, '0.13', 'elapsed: ', '0:00:13.454741')
(12, '0.14', 'elapsed: ', '0:00:14.636362')
(13, '0.15', 'elapsed: ', '0:00:15.867308')
(14, '0.16', 'elapsed: ', '0:00:17.052000')
(15, '0.18', 'elapsed: ', '0:00:18.263081')
(16, '0.19', 'elapsed: ', '0:00:19.442278')
(17, '0.20', 'elapsed: ', '0:00:20.724541')
(18, '0.21', 'elapsed: ', '0:00:21.939463')
(19, '0.22', 'elapsed: ', '0:00:23.145336')
(20, '0.23', 'elapsed: ', '0:00:24.321772')
(21, '0.25', 'elapsed: ', '0:00:25.679418')
(22, '0.26', 'elapsed: ', '0:00:26.902407'

(185, '2.16', 'elapsed: ', '0:03:54.705049')
(186, '2.18', 'elapsed: ', '0:03:55.942277')
(187, '2.19', 'elapsed: ', '0:03:57.182309')
(188, '2.20', 'elapsed: ', '0:03:58.377042')
(189, '2.21', 'elapsed: ', '0:03:59.508545')
(190, '2.22', 'elapsed: ', '0:04:00.654990')
(191, '2.23', 'elapsed: ', '0:04:01.778858')
(192, '2.25', 'elapsed: ', '0:04:02.933879')
(193, '2.26', 'elapsed: ', '0:04:04.168075')
(194, '2.27', 'elapsed: ', '0:04:05.509787')
(195, '2.28', 'elapsed: ', '0:04:06.723573')
(196, '2.29', 'elapsed: ', '0:04:07.876886')
(197, '2.30', 'elapsed: ', '0:04:09.061529')
(198, '2.32', 'elapsed: ', '0:04:10.248158')
(199, '2.33', 'elapsed: ', '0:04:11.372308')
(200, '2.34', 'elapsed: ', '0:04:12.525439')
(201, '2.35', 'elapsed: ', '0:04:13.781027')
(202, '2.36', 'elapsed: ', '0:04:15.077334')
(203, '2.37', 'elapsed: ', '0:04:16.314380')
(204, '2.39', 'elapsed: ', '0:04:18.346662')
(205, '2.40', 'elapsed: ', '0:04:19.507800')
(206, '2.41', 'elapsed: ', '0:04:20.700270')
(207, '2.4

(368, '4.31', 'elapsed: ', '0:07:39.584283')
(369, '4.32', 'elapsed: ', '0:07:40.635193')
(370, '4.33', 'elapsed: ', '0:07:41.843156')
(371, '4.34', 'elapsed: ', '0:07:43.041147')
(372, '4.35', 'elapsed: ', '0:07:44.225310')
(373, '4.36', 'elapsed: ', '0:07:45.363888')
(374, '4.38', 'elapsed: ', '0:07:46.483765')
(375, '4.39', 'elapsed: ', '0:07:47.609858')
(376, '4.40', 'elapsed: ', '0:07:48.913088')
(377, '4.41', 'elapsed: ', '0:07:50.046479')
(378, '4.42', 'elapsed: ', '0:07:51.174714')
(379, '4.43', 'elapsed: ', '0:07:52.334711')
(380, '4.45', 'elapsed: ', '0:07:53.409169')
(381, '4.46', 'elapsed: ', '0:07:54.536141')
(382, '4.47', 'elapsed: ', '0:07:55.702590')
(383, '4.48', 'elapsed: ', '0:07:56.859703')
(384, '4.49', 'elapsed: ', '0:07:58.017194')
(385, '4.50', 'elapsed: ', '0:07:59.148957')
(386, '4.52', 'elapsed: ', '0:08:00.338526')
(387, '4.53', 'elapsed: ', '0:08:01.553684')
(388, '4.54', 'elapsed: ', '0:08:04.187419')
(389, '4.55', 'elapsed: ', '0:08:05.287594')
(390, '4.5

ERROR!!
/opt/anaconda3/bin/earthengine upload image --asset_id projects/WRI-Aquaduct/PCRGlobWB20V05/global_historical_PDomWN_month_millionm3_5min_1960_2014/global_historical_PDomWN_month_millionm3_5min_1960_2014I550Y2005M11 gs://aqueduct30_v01/Y2017M08D02_RH_Upload_to_GoogleCS_V01/global_historical_PDomWN_month_millionm3_5min_1960_2014I550Y2005M11.tif --nodata_value=-9999 --time_start 2005-11-01 -p extension=tif -p filename=global_historical_PDomWN_month_millionm3_5min_1960_2014I550Y2005M11 -p identifier=550 -p year=2005 -p geographic_range=global -p indicator=PDomWN -p spatial_resolution=5min -p temporal_range=historical -p temporal_range_max=2014 -p temporal_range_min=1960 -p temporal_resolution=month -p units=millionm3 -p ingested_by=RutgerHofste -p exportdescription=PDomWN_monthY2005M11
(551, '6.45', 'elapsed: ', '0:11:41.415064')
(552, '6.46', 'elapsed: ', '0:11:42.608034')
(553, '6.47', 'elapsed: ', '0:11:43.868477')
(554, '6.48', 'elapsed: ', '0:11:44.991178')
(555, '6.49', 'ela

(716, '8.38', 'elapsed: ', '0:15:19.899428')
(717, '8.39', 'elapsed: ', '0:15:21.119360')
(718, '8.40', 'elapsed: ', '0:15:22.246704')
(719, '8.41', 'elapsed: ', '0:15:23.375158')
(720, '8.42', 'elapsed: ', '0:15:24.551284')
(721, '8.43', 'elapsed: ', '0:15:25.921615')
(722, '8.45', 'elapsed: ', '0:15:27.188280')
(723, '8.46', 'elapsed: ', '0:15:28.655872')
(724, '8.47', 'elapsed: ', '0:15:29.809597')
(725, '8.48', 'elapsed: ', '0:15:31.030945')
(726, '8.49', 'elapsed: ', '0:15:32.228487')
(727, '8.50', 'elapsed: ', '0:15:33.958043')
(728, '8.52', 'elapsed: ', '0:15:35.059637')
(729, '8.53', 'elapsed: ', '0:15:36.212875')
(730, '8.54', 'elapsed: ', '0:15:37.462778')
(731, '8.55', 'elapsed: ', '0:15:39.614369')
(732, '8.56', 'elapsed: ', '0:15:40.759927')
(733, '8.58', 'elapsed: ', '0:15:41.996512')
(734, '8.59', 'elapsed: ', '0:15:43.383773')
(735, '8.60', 'elapsed: ', '0:15:44.484552')
(736, '8.61', 'elapsed: ', '0:15:45.728571')
(737, '8.62', 'elapsed: ', '0:15:46.971046')
(738, '8.6

...

(1758, '20.57', 'elapsed: ', '0:38:33.254335')
(1759, '20.58', 'elapsed: ', '0:38:34.609308')
(1760, '20.59', 'elapsed: ', '0:38:35.697896')
(1761, '20.60', 'elapsed: ', '0:38:36.882604')
(1762, '20.61', 'elapsed: ', '0:38:37.957992')
(1763, '20.62', 'elapsed: ', '0:38:39.121768')
(1764, '20.64', 'elapsed: ', '0:38:40.276197')
(1765, '20.65', 'elapsed: ', '0:38:41.367207')
(1766, '20.66', 'elapsed: ', '0:38:42.472118')
(1767, '20.67', 'elapsed: ', '0:38:43.598124')
(1768, '20.68', 'elapsed: ', '0:38:44.741109')
(1769, '20.69', 'elapsed: ', '0:38:45.881234')
(1770, '20.71', 'elapsed: ', '0:38:47.109328')
(1771, '20.72', 'elapsed: ', '0:38:48.216036')
(1772, '20.73', 'elapsed: ', '0:38:49.316547')
(1773, '20.74', 'elapsed: ', '0:38:50.508510')
(1774, '20.75', 'elapsed: ', '0:38:51.655025')
(1775, '20.77', 'elapsed: ', '0:38:52.774420')
(1776, '20.78', 'elapsed: ', '0:38:53.908128')
(1777, '20.79', 'elapsed: ', '0:38:55.028601')
(1778, '20.80', 'elapsed: ', '0:38:56.173445')
(1779, '20.81

(1933, '22.61', 'elapsed: ', '0:41:57.585337')
(1934, '22.63', 'elapsed: ', '0:41:58.753773')
(1935, '22.64', 'elapsed: ', '0:41:59.885477')
(1936, '22.65', 'elapsed: ', '0:42:03.411350')
(1937, '22.66', 'elapsed: ', '0:42:04.727650')
(1938, '22.67', 'elapsed: ', '0:42:05.828545')
(1939, '22.68', 'elapsed: ', '0:42:07.032424')
(1940, '22.70', 'elapsed: ', '0:42:08.294862')
(1941, '22.71', 'elapsed: ', '0:42:09.433233')
(1942, '22.72', 'elapsed: ', '0:42:10.564338')
(1943, '22.73', 'elapsed: ', '0:42:15.000881')
(1944, '22.74', 'elapsed: ', '0:42:16.117921')
(1945, '22.75', 'elapsed: ', '0:42:17.291069')
(1946, '22.77', 'elapsed: ', '0:42:18.394607')
(1947, '22.78', 'elapsed: ', '0:42:19.534296')
(1948, '22.79', 'elapsed: ', '0:42:20.650328')
(1949, '22.80', 'elapsed: ', '0:42:21.762251')
(1950, '22.81', 'elapsed: ', '0:42:22.884696')
(1951, '22.82', 'elapsed: ', '0:42:24.010102')
(1952, '22.84', 'elapsed: ', '0:42:25.117735')
(1953, '22.85', 'elapsed: ', '0:42:26.215539')
(1954, '22.86

(2108, '24.66', 'elapsed: ', '0:45:41.847631')
(2109, '24.67', 'elapsed: ', '0:45:43.052067')
(2110, '24.68', 'elapsed: ', '0:45:44.538381')
(2111, '24.70', 'elapsed: ', '0:45:45.698819')
(2112, '24.71', 'elapsed: ', '0:45:46.812320')
(2113, '24.72', 'elapsed: ', '0:45:47.932758')
(2114, '24.73', 'elapsed: ', '0:45:49.024947')
(2115, '24.74', 'elapsed: ', '0:45:50.175353')
(2116, '24.75', 'elapsed: ', '0:45:51.285589')
(2117, '24.77', 'elapsed: ', '0:45:52.468398')
(2118, '24.78', 'elapsed: ', '0:45:53.763828')
(2119, '24.79', 'elapsed: ', '0:45:54.894169')
(2120, '24.80', 'elapsed: ', '0:45:56.042039')
(2121, '24.81', 'elapsed: ', '0:45:57.157934')
(2122, '24.82', 'elapsed: ', '0:45:58.200059')
(2123, '24.84', 'elapsed: ', '0:45:59.421883')
(2124, '24.85', 'elapsed: ', '0:46:00.534367')
(2125, '24.86', 'elapsed: ', '0:46:01.906789')
(2126, '24.87', 'elapsed: ', '0:46:03.029065')
(2127, '24.88', 'elapsed: ', '0:46:04.238785')
(2128, '24.89', 'elapsed: ', '0:46:05.334611')
(2129, '24.91

(2283, '26.71', 'elapsed: ', '0:49:14.631131')
(2284, '26.72', 'elapsed: ', '0:49:16.253514')
(2285, '26.73', 'elapsed: ', '0:49:17.298117')
(2286, '26.74', 'elapsed: ', '0:49:18.415760')
(2287, '26.75', 'elapsed: ', '0:49:19.659103')
(2288, '26.77', 'elapsed: ', '0:49:20.824764')
(2289, '26.78', 'elapsed: ', '0:49:22.009257')
(2290, '26.79', 'elapsed: ', '0:49:23.162959')
(2291, '26.80', 'elapsed: ', '0:49:24.299236')
(2292, '26.81', 'elapsed: ', '0:49:25.482812')
(2293, '26.82', 'elapsed: ', '0:49:26.652599')
(2294, '26.84', 'elapsed: ', '0:49:27.800901')
(2295, '26.85', 'elapsed: ', '0:49:28.933675')
(2296, '26.86', 'elapsed: ', '0:49:30.005808')
(2297, '26.87', 'elapsed: ', '0:49:31.088278')
(2298, '26.88', 'elapsed: ', '0:49:32.181201')
(2299, '26.90', 'elapsed: ', '0:49:33.274814')
(2300, '26.91', 'elapsed: ', '0:49:34.442092')
(2301, '26.92', 'elapsed: ', '0:49:35.604186')
(2302, '26.93', 'elapsed: ', '0:49:37.838108')
(2303, '26.94', 'elapsed: ', '0:49:38.939507')
(2304, '26.95

(2451, '28.67', 'elapsed: ', '0:52:53.774543')
(2452, '28.69', 'elapsed: ', '0:52:54.930067')
(2453, '28.70', 'elapsed: ', '0:52:56.038848')
(2454, '28.71', 'elapsed: ', '0:52:57.089730')
(2455, '28.72', 'elapsed: ', '0:52:58.192327')
(2456, '28.73', 'elapsed: ', '0:52:59.406469')
(2457, '28.74', 'elapsed: ', '0:53:00.588093')
(2458, '28.76', 'elapsed: ', '0:53:04.034258')
(2459, '28.77', 'elapsed: ', '0:53:05.274469')
(2460, '28.78', 'elapsed: ', '0:53:06.373856')
(2461, '28.79', 'elapsed: ', '0:53:07.531337')
(2462, '28.80', 'elapsed: ', '0:53:08.643552')
(2463, '28.81', 'elapsed: ', '0:53:09.779722')
(2464, '28.83', 'elapsed: ', '0:53:10.938606')
(2465, '28.84', 'elapsed: ', '0:53:12.034270')
(2466, '28.85', 'elapsed: ', '0:53:13.151431')
(2467, '28.86', 'elapsed: ', '0:53:14.341354')
(2468, '28.87', 'elapsed: ', '0:53:15.669410')
(2469, '28.88', 'elapsed: ', '0:53:18.021466')
(2470, '28.90', 'elapsed: ', '0:53:20.139039')
(2471, '28.91', 'elapsed: ', '0:53:21.272556')
(2472, '28.92